# Testing basic LSTM
### creating mock dataset 

In [20]:
# create sample data
# try to learn fruit names letter-wise

words = ['apple', 'banana', 'pear', 'lime', 'cherry', 'orange', 'berry', 'strawberry']

# sequence stop sign
SEQUENCE_STOP_SIGN = '.'

# transform words to letter arrays
words_arrays = [[l for l in w] for w in words]

In [40]:
# create sequences in many-to-many configuration
sequences_x, sequences_y = zip(*[(w, w[1:] + [SEQUENCE_STOP_SIGN]) for w in words_arrays])
sequences_x[0], sequences_y[0]

(['a', 'p', 'p', 'l', 'e'], ['p', 'p', 'l', 'e', '.'])

In [32]:
# one-hot encode letters (and stop sign)
import string
ALPHABET = string.ascii_lowercase + SEQUENCE_STOP_SIGN
ALPHABET_LENGTH = len(ALPHABET)

import numpy as np
def encode_letter(letter):
    res = np.zeros(ALPHABET_LENGTH)
    res[ALPHABET.index(letter)] = 1
    return res

def decode_letter(ohe_vec):
    # for now, one dimention only
    return ALPHABET[ohe_vec.argmax()]

encode_letter('d')

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [39]:
def encode_word(word):
    return np.stack([encode_letter(l) for l in word])
    
def decode_word(ohe_word):
    return ''.join([decode_letter(ohe_letter) for ohe_letter in ohe_word])

encode_word('asd')

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [48]:
# encode dataset
data_x = [encode_word(seq) for seq in sequences_x]
data_y = [encode_word(seq) for seq in sequences_y]

In [70]:
# data generator
import random
def data_gen():
    while True:
        index = random.randint(0, len(data_x) - 1)
        yield (
            np.expand_dims(data_x[index], axis=0), 
            np.expand_dims(data_y[index], axis=0)
        )

### create model

In [85]:
import tensorflow as tf

INPUT_UNITS = ALPHABET_LENGTH
HIDDEN_UNITS = 128
OUTPUT_UNITS = ALPHABET_LENGTH

# (seq_len, input)
INPUT_SHAPE = (None, INPUT_UNITS)

model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(HIDDEN_UNITS, input_shape=INPUT_SHAPE, return_sequences=True),
    tf.keras.layers.Dense(OUTPUT_UNITS, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam', 
    metrics=['categorical_accuracy']
)


In [86]:
gen = data_gen()
STEPS_PER_EPOCH = len(data_x)
EPOCHS = 100

model.fit_generator(gen, steps_per_epoch=STEPS_PER_EPOCH, epochs=EPOCHS)

Epoch 1/100
8/8 [==============================] - 3s 388ms/step - loss: 3.2574 - categorical_accuracy: 0.2500
Epoch 2/100
8/8 [==============================] - 0s 13ms/step - loss: 3.2287 - categorical_accuracy: 0.2792
Epoch 3/100
8/8 [==============================] - 0s 14ms/step - loss: 3.1269 - categorical_accuracy: 0.3875
Epoch 4/100
8/8 [==============================] - 0s 15ms/step - loss: 3.0105 - categorical_accuracy: 0.2042
Epoch 5/100
8/8 [==============================] - 0s 13ms/step - loss: 2.7121 - categorical_accuracy: 0.2854
Epoch 6/100
8/8 [==============================] - 0s 13ms/step - loss: 2.5335 - categorical_accuracy: 0.2208
Epoch 7/100
8/8 [==============================] - 0s 13ms/step - loss: 2.4110 - categorical_accuracy: 0.2021
Epoch 8/100
8/8 [==============================] - 0s 14ms/step - loss: 2.4825 - categorical_accuracy: 0.2896
Epoch 9/100
8/8 [==============================] - 0s 14ms/step - loss: 2.3511 - categorical_accuracy: 0.3500
Epoch 10/

8/8 [==============================] - 0s 13ms/step - loss: 0.4025 - categorical_accuracy: 0.9458
Epoch 74/100
8/8 [==============================] - 0s 14ms/step - loss: 0.3872 - categorical_accuracy: 1.0000
Epoch 75/100
8/8 [==============================] - 0s 18ms/step - loss: 0.4051 - categorical_accuracy: 0.9542
Epoch 76/100
8/8 [==============================] - 0s 22ms/step - loss: 0.4359 - categorical_accuracy: 0.9792
Epoch 77/100
8/8 [==============================] - 0s 15ms/step - loss: 0.3391 - categorical_accuracy: 0.9583
Epoch 78/100
8/8 [==============================] - ETA: 0s - loss: 0.3851 - categorical_accuracy: 0.90 - 0s 20ms/step - loss: 0.4160 - categorical_accuracy: 0.9292
Epoch 79/100
8/8 [==============================] - 0s 21ms/step - loss: 0.3432 - categorical_accuracy: 0.9333
Epoch 80/100
8/8 [==============================] - 0s 14ms/step - loss: 0.3572 - categorical_accuracy: 0.9333
Epoch 81/100
8/8 [==============================] - 0s 13ms/step - loss

In [82]:
prediction = model.predict(np.expand_dims(encode_word('pe'), axis=0))
decode_word(np.squeeze(prediction, axis=0))

'ea'